In [ ]:
!pip install avalanche-lib medmnist torch torchvision --quiet

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import transforms
from torch.utils.data import Dataset

import medmnist
from medmnist import PathMNIST

from avalanche.benchmarks import nc_benchmark
from avalanche.training import EWC
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    forgetting_metrics
)
from avalanche.logging import InteractiveLogger


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [4]:
#Dataset wrapper
class MedMNISTWrapper(Dataset):
    def __init__(self, medmnist_dataset):
        self.dataset = medmnist_dataset
        self.targets = [
            int(label) for label in medmnist_dataset.labels.squeeze()
        ]

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        x, y = self.dataset[index]
        return x, int(y)

In [5]:
#Load PathMNIST
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_raw = PathMNIST(
    split="train",
    transform=transform,
    download=True
)

test_raw = PathMNIST(
    split="test",
    transform=transform,
    download=True
)

train_dataset = MedMNISTWrapper(train_raw)
test_dataset  = MedMNISTWrapper(test_raw)


100%|██████████| 206M/206M [00:11<00:00, 18.0MB/s]


In [6]:
benchmark = nc_benchmark(
    train_dataset,
    test_dataset,
    n_experiences=3,
    task_labels=False,
    shuffle=True,
    seed=123
)


In [7]:
print("Total classes:", benchmark.n_classes)

for exp in benchmark.train_stream:
    print(
        f"Experience {exp.current_experience}:",
        exp.classes_in_this_experience
    )


Total classes: 9
Experience 0: [2, 4, 6]
Experience 1: [0, 3, 7]
Experience 2: [8, 1, 5]


In [8]:
#CNN model
class CNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Linear(64 * 7 * 7, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

In [9]:
model = CNN(num_classes=benchmark.n_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
#optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()


In [10]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(epoch=True, experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    loggers=[InteractiveLogger()]
)

In [11]:
cl_strategy = EWC(
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    ewc_lambda=5000,          # strength of regularization
    train_mb_size=64,
    train_epochs=8,
    eval_mb_size=128,
    device=device,
    evaluator=eval_plugin
)


In [13]:
for experience in benchmark.train_stream:
    print("Training on experience", experience.current_experience)
    cl_strategy.train(experience)

    print("Evaluating...")
    cl_strategy.eval(benchmark.test_stream)

Training on experience 0
-- >> Start of training phase << --
 26%|██▋       | 108/411 [00:16<01:59,  2.53it/s]

/tmp/ipython-input-1271679127.py:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return x, int(y)


506it [00:21, 23.03it/s]
Epoch 0 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9464
100%|██████████| 411/411 [00:06<00:00, 62.82it/s]
Epoch 1 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9485
100%|██████████| 411/411 [00:06<00:00, 60.29it/s]
Epoch 2 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9559
100%|██████████| 411/411 [00:06<00:00, 60.64it/s]
Epoch 3 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9653
100%|██████████| 411/411 [00:06<00:00, 66.28it/s]
Epoch 4 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9669
100%|██████████| 411/411 [00:06<00:00, 61.96it/s]
Epoch 5 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9693
100%|██████████| 411/411 [00:06<00:00, 59.86it/s]
Epoch 6 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9683
100%|██████████| 411/411 [00:06<00:00, 62.12it/s]
Epoch 7 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9724
-- >> End of training phase << --
Evaluat

In [14]:
import json

metrics = cl_strategy.evaluator.get_all_metrics()

with open("ewc_metrics.json", "w") as f:
    json.dump(metrics, f)
